In [1]:
# Importing necessary libraries
import numpy as np
import librosa
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# Define function to extract features from audio
def extract_features(file_path):
    try:
        audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30) 
        mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=13)
        mfccs_processed = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_path)
        return None 
     
    return mfccs_processed

In [3]:
# Define function to load data and extract features
def load_data(data_path):
    labels = []
    features = []
    
    for folder in os.listdir(data_path):
        genre_folder = os.path.join(data_path, folder)
        for file in os.listdir(genre_folder):
            file_path = os.path.join(genre_folder, file)
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(folder)
                
    return np.array(features), np.array(labels)

In [4]:
# Load data
data_path = 'Data/genres_original'  # Update with your dataset path
features, labels = load_data(data_path)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(encoded_labels)

C:\Users\JLCarunungan\AppData\Local\Temp\ipykernel_14044\2273535198.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30)
c:\Users\JLCarunungan\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error encountered while parsing file:  Data/genres_original\jazz\jazz.00054.wav


In [5]:
features.shape

(999, 13)

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_categorical, test_size=0.2, random_state=42)

In [7]:
X_train.shape[0]

799

In [8]:
# Reshape input data to have the appropriate shape for RNN
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [9]:
# Define the model
model = Sequential()
model.add(SimpleRNN(units=128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))  # Adjust dropout rate
model.add(BatchNormalization())  # Add BatchNormalization layer
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

c:\Users\JLCarunungan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/30
19/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2012 - loss: 2.5664
Epoch 1: val_loss improved from inf to 2.05368, saving model to best_model.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.2110 - loss: 2.5022 - val_accuracy: 0.2400 - val_loss: 2.0537
Epoch 2/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2906 - loss: 2.0457
Epoch 2: val_loss improved from 2.05368 to 1.89398, saving model to best_model.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2929 - loss: 2.0423 - val_accuracy: 0.3250 - val_loss: 1.8940
Epoch 3/30
23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4121 - loss: 1.7063
Epoch 3: val_loss improved from 1.89398 to 1.86076, saving model to best_model.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4074 - loss: 1.7188 - val_accuracy: 0.3500 - val_loss: 1.8608
Epoch 4/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3952 - loss: 1.7291
Epoch 4: val_loss improved from 1.86076 to 1.80895, saving model to

In [11]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model
model.save('music_genre_classifier_rnn.keras')
print("Model saved as 'music_genre_classifier_rnn.keras'")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4397 - loss: 1.6597 
Test Loss: 1.5585402250289917
Test Accuracy: 0.4650000035762787
Model saved as 'music_genre_classifier_rnn.keras'
